In [1]:
import torch
import pandas as pd
from torchmetrics.text import ROUGEScore
from transformers import BartForConditionalGeneration, BartTokenizerFast
from tqdm.notebook import tqdm

In [3]:
df = pd.read_csv("../../../data/labelled/metadata/splits/test.csv")

In [4]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

device

'cuda'

In [6]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [7]:
%%capture
model.to(device)

In [8]:
tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-large-cnn")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [2]:
rouge = ROUGEScore()

In [10]:
batch_size = 8

In [11]:
rouge_scores = {"rouge1":[], "rouge2":[], "rougeL": []}
for idx in tqdm(range(0, len(df), batch_size)):
    batch = df[idx:idx+batch_size]
    product_descriptions = batch.product_info.to_list()
    actual_summary = batch.summary.to_list()

    input_ids = tokenizer(product_descriptions, max_length=1024, return_tensors="pt", padding=True, truncation=True)["input_ids"].to(device)
    summary_ids = model.generate(input_ids, min_length=0, max_length=512)
    preds = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

    for pred, target in zip(preds, actual_summary):
        rouge_all = rouge(pred, target)
        rouge_scores['rouge1'].append(rouge_all['rouge1_fmeasure'].item())
        rouge_scores['rouge2'].append(rouge_all['rouge2_fmeasure'].item())
        rouge_scores['rougeL'].append(rouge_all['rougeL_fmeasure'].item())


  0%|          | 0/3136 [00:00<?, ?it/s]

In [12]:
df['rouge1'] = rouge_scores['rouge1']
df['rouge2'] = rouge_scores['rouge2']
df['rougeL'] = rouge_scores['rougeL']

In [13]:
df.to_csv("benchmark_scores.csv", index=False)

In [9]:
df = pd.read_csv("benchmark_scores.csv")

In [3]:
df.head()

,product_info,summary,rouge1,rouge2,rougeL
0,<b>CONTENTS - In the Friendly Swede Retail pa...,The Friendly Swede Retail packaging includes a...,0.185714,0.043478,0.128571
1,EMPIRE's two piece snap-on cases are made of h...,EMPIRE's two piece snap-on cases provide maxim...,0.583333,0.342857,0.472222
2,This protector case is custom made for your de...,"A custom-made protector case for your device, ...",0.492308,0.253968,0.369231
3,"120 Volt 15 Amp Surge Protected, 19 Inch Rack ...",Surge-protected power distribution unit for 19...,0.406250,0.129032,0.250000
4,"Your digital data, photos, videos, email, docu...",The Maxtor OneTouch III provides an automated ...,0.297872,0.065217,0.212766


In [6]:
scores = {"rouge1": df.rouge1.mean(), "rouge2": df.rouge2.mean(), "rougeL": df.rougeL.mean()}

In [7]:
scores

{'rouge1': 0.4332254241812763,
 'rouge2': 0.24094070826252378,
 'rougeL': 0.3469129775269041}

In [18]:
summaries = df.summary.to_list()
lengths = list(map(len, list(map(str.split, summaries))))
sum(lengths)/len(lengths)

53.35749023359643

In [19]:
product_infos = df.product_info.to_list()
lengths = list(map(len, list(map(str.split, product_infos))))
sum(lengths)/len(lengths)

160.55708363230488